In [4]:
import pandas as pd
import datetime 
import requests
from io import StringIO
from bs4 import BeautifulSoup
from lxml import html

### URL para 0kms: https://autos.mercadolibre.com.ar/0-km/okm_Desde_1_NoIndex_True y URL para usados: https://autos.mercadolibre.com.ar/usados-autos_Desde_1_NoIndex_True

In [5]:
okm_url = "https://autos.mercadolibre.com.ar/0-km/okm_NoIndex_True#applied_filter_id%3DKILOMETERS%26applied_filter_name%3DKil%C3%B3metros%26applied_filter_order%3D4%26applied_value_id%3D%5B0km-0km%5D%26applied_value_name%3D0+km%26applied_value_order%3D2%26applied_value_results%3D531%26is_custom%3Dfalse"
usados_url = "https://autos.mercadolibre.com.ar/usados-autos_ITEM*CONDITION_2230581_NoIndex_True#applied_filter_id%3DITEM_CONDITION%26applied_filter_name%3DCondici%C3%B3n%26applied_filter_order%3D5%26applied_value_id%3D2230581%26applied_value_name%3DUsado%26applied_value_order%3D2%26applied_value_results%3D506%26is_custom%3Dfalse"

In [6]:
fecha_hora_actual = datetime.datetime.now()
# Formatear la fecha y hora en un formato específico
formato_personalizado = "%Y-%m-%d %H:%M:%S"  # Por ejemplo, "2023-08-25 15:30:00"
fecha_hora_formateada = fecha_hora_actual.strftime(formato_personalizado)
print("Fecha y hora:", fecha_hora_formateada)

Fecha y hora: 2023-08-27 19:06:30


### obtener las url de los 0kms

In [7]:
okm_links = []
i = 0
base_url = 'https://autos.mercadolibre.com.ar/0-km/okm_Desde_{page}_NoIndex_True'

while True:
    url = base_url.format(page=i + 1)
    print(url)
    res = requests.get(url)
    
    if res.status_code != 200:
        break
        
    soup = BeautifulSoup(res.text, "html.parser")
    div_links_okm = soup.find_all('div', class_='ui-search-item__group ui-search-item__group--title shops__items-group')
    links_okm = [div.find('a')['href'] for div in div_links_okm]
    nombres_okm = [link.find('h2').get_text(strip=True) if link.find('h2') else '' for link in div_links_okm]
    
    for link, nom in list(zip(links_okm, nombres_okm)):
#         print({'type': '0km', 'link_web': link, 'nom': nom})
        model_dict = {
            'type': '0km',
            'link_web': link,
            'modelo': nom
        }
        okm_links.append(model_dict)
    
    i += 48
# okm_links

https://autos.mercadolibre.com.ar/0-km/okm_Desde_1_NoIndex_True
https://autos.mercadolibre.com.ar/0-km/okm_Desde_49_NoIndex_True
https://autos.mercadolibre.com.ar/0-km/okm_Desde_97_NoIndex_True
https://autos.mercadolibre.com.ar/0-km/okm_Desde_145_NoIndex_True
https://autos.mercadolibre.com.ar/0-km/okm_Desde_193_NoIndex_True
https://autos.mercadolibre.com.ar/0-km/okm_Desde_241_NoIndex_True
https://autos.mercadolibre.com.ar/0-km/okm_Desde_289_NoIndex_True
https://autos.mercadolibre.com.ar/0-km/okm_Desde_337_NoIndex_True
https://autos.mercadolibre.com.ar/0-km/okm_Desde_385_NoIndex_True
https://autos.mercadolibre.com.ar/0-km/okm_Desde_433_NoIndex_True
https://autos.mercadolibre.com.ar/0-km/okm_Desde_481_NoIndex_True
https://autos.mercadolibre.com.ar/0-km/okm_Desde_529_NoIndex_True


In [8]:
df_okm_model_links = pd.DataFrame(okm_links)
df_okm_model_links

,type,link_web,modelo
0,0km,https://auto.mercadolibre.com.ar/MLA-147437521...,Peugeot 208 1.2 New Like
1,0km,https://auto.mercadolibre.com.ar/MLA-138489725...,Volkswagen Taos 1.4 250 Tsi Comfortline
2,0km,https://auto.mercadolibre.com.ar/MLA-148042873...,Volkswagen Amarok 3.0 V6 Extreme
3,0km,https://auto.mercadolibre.com.ar/MLA-148042856...,Volkswagen Amarok 3.0 V6 Cd Highline
4,0km,https://auto.mercadolibre.com.ar/MLA-138449856...,Volkswagen Amarok 3.0 V6 Cd Comfortline
...,...,...,...
521,0km,https://auto.mercadolibre.com.ar/MLA-138196186...,Mercedes-benz Sprinter 316 Mixto 4+1 3665 Te
522,0km,https://auto.mercadolibre.com.ar/MLA-145331258...,Chery Tiggo 4 Pro 1.5 Comfort Mt
523,0km,https://auto.mercadolibre.com.ar/MLA-144144665...,Toyota Hilux 2.8 Cd Srx 177cv 4x4 At
524,0km,https://auto.mercadolibre.com.ar/MLA-925824993...,Foton Aumark Tm1 C/d


### obtener las url de los usados

In [9]:
usados_links = []
i = 0
base_url = 'https://autos.mercadolibre.com.ar/usados-autos_Desde_{page}_ITEM*CONDITION_2230581_NoIndex_True'

while True:
    url = base_url.format(page=i + 1)
    print(url)
    res = requests.get(url)
    
    if res.status_code != 200:
        break
        
    soup = BeautifulSoup(res.text, "html.parser")
    div_links_usados = soup.find_all('div', class_='ui-search-item__group ui-search-item__group--title shops__items-group')
    links_usados = [div.find('a')['href'] for div in div_links_usados]
    nombres_usados = [link.find('h2').get_text(strip=True) if link.find('h2') else '' for link in div_links_usados]

    for link, nom in list(zip(links_usados, nombres_usados)):
#         print({'type': 'usado', 'link_web': link, 'nom': nom})
        model_dict = {
            'type': 'usado',
            'link_web': link,
            'modelo': nom
        }
        usados_links.append(model_dict)
    
    i += 48
# usados_links

https://autos.mercadolibre.com.ar/usados-autos_Desde_1_ITEM*CONDITION_2230581_NoIndex_True
https://autos.mercadolibre.com.ar/usados-autos_Desde_49_ITEM*CONDITION_2230581_NoIndex_True
https://autos.mercadolibre.com.ar/usados-autos_Desde_97_ITEM*CONDITION_2230581_NoIndex_True
https://autos.mercadolibre.com.ar/usados-autos_Desde_145_ITEM*CONDITION_2230581_NoIndex_True
https://autos.mercadolibre.com.ar/usados-autos_Desde_193_ITEM*CONDITION_2230581_NoIndex_True
https://autos.mercadolibre.com.ar/usados-autos_Desde_241_ITEM*CONDITION_2230581_NoIndex_True
https://autos.mercadolibre.com.ar/usados-autos_Desde_289_ITEM*CONDITION_2230581_NoIndex_True
https://autos.mercadolibre.com.ar/usados-autos_Desde_337_ITEM*CONDITION_2230581_NoIndex_True
https://autos.mercadolibre.com.ar/usados-autos_Desde_385_ITEM*CONDITION_2230581_NoIndex_True
https://autos.mercadolibre.com.ar/usados-autos_Desde_433_ITEM*CONDITION_2230581_NoIndex_True
https://autos.mercadolibre.com.ar/usados-autos_Desde_481_ITEM*CONDITION_22

In [10]:
df_usados_model_links = pd.DataFrame(usados_links)
df_usados_model_links

,type,link_web,modelo
0,usado,https://auto.mercadolibre.com.ar/MLA-148356516...,Chevrolet Onix 1.4 Ltz Mt 98cv
1,usado,https://auto.mercadolibre.com.ar/MLA-138488314...,Renault Fluence 2.0 Privilege Cvt
2,usado,https://auto.mercadolibre.com.ar/MLA-138490950...,Renault Fluence 2.0 Privilege Cvt
3,usado,https://auto.mercadolibre.com.ar/MLA-138490876...,Ford Focus Ii 2.0 Exe Sedan Ghia
4,usado,https://auto.mercadolibre.com.ar/MLA-138493666...,Ford Ka 1.5 S 4 P
...,...,...,...
531,usado,https://auto.mercadolibre.com.ar/MLA-137853497...,Peugeot 308 1.6 Allure Nav 115cv
532,usado,https://auto.mercadolibre.com.ar/MLA-144093409...,Chevrolet Cruze 1.4 Ltz At Sedan
533,usado,https://auto.mercadolibre.com.ar/MLA-144740810...,Jeep Renegade 1.8 Sport
534,usado,https://auto.mercadolibre.com.ar/MLA-137762034...,Volkswagen Amarok 3.0 V6 Extreme


In [11]:
df_mercado_libre_links = pd.concat([df_okm_model_links, df_usados_model_links], ignore_index=True)
df_mercado_libre_links

,type,link_web,modelo
0,0km,https://auto.mercadolibre.com.ar/MLA-147437521...,Peugeot 208 1.2 New Like
1,0km,https://auto.mercadolibre.com.ar/MLA-138489725...,Volkswagen Taos 1.4 250 Tsi Comfortline
2,0km,https://auto.mercadolibre.com.ar/MLA-148042873...,Volkswagen Amarok 3.0 V6 Extreme
3,0km,https://auto.mercadolibre.com.ar/MLA-148042856...,Volkswagen Amarok 3.0 V6 Cd Highline
4,0km,https://auto.mercadolibre.com.ar/MLA-138449856...,Volkswagen Amarok 3.0 V6 Cd Comfortline
...,...,...,...
1057,usado,https://auto.mercadolibre.com.ar/MLA-137853497...,Peugeot 308 1.6 Allure Nav 115cv
1058,usado,https://auto.mercadolibre.com.ar/MLA-144093409...,Chevrolet Cruze 1.4 Ltz At Sedan
1059,usado,https://auto.mercadolibre.com.ar/MLA-144740810...,Jeep Renegade 1.8 Sport
1060,usado,https://auto.mercadolibre.com.ar/MLA-137762034...,Volkswagen Amarok 3.0 V6 Extreme


In [12]:
df_mercado_libre_links.to_csv('../CSV/df_mercado_libre_model_links.csv', index=False)

In [13]:
df_mercado_libre_links = pd.read_csv('../CSV/df_mercado_libre_model_links.csv')
df_mercado_libre_links

,type,link_web,modelo
0,0km,https://auto.mercadolibre.com.ar/MLA-147437521...,Peugeot 208 1.2 New Like
1,0km,https://auto.mercadolibre.com.ar/MLA-138489725...,Volkswagen Taos 1.4 250 Tsi Comfortline
2,0km,https://auto.mercadolibre.com.ar/MLA-148042873...,Volkswagen Amarok 3.0 V6 Extreme
3,0km,https://auto.mercadolibre.com.ar/MLA-148042856...,Volkswagen Amarok 3.0 V6 Cd Highline
4,0km,https://auto.mercadolibre.com.ar/MLA-138449856...,Volkswagen Amarok 3.0 V6 Cd Comfortline
...,...,...,...
1057,usado,https://auto.mercadolibre.com.ar/MLA-137853497...,Peugeot 308 1.6 Allure Nav 115cv
1058,usado,https://auto.mercadolibre.com.ar/MLA-144093409...,Chevrolet Cruze 1.4 Ltz At Sedan
1059,usado,https://auto.mercadolibre.com.ar/MLA-144740810...,Jeep Renegade 1.8 Sport
1060,usado,https://auto.mercadolibre.com.ar/MLA-137762034...,Volkswagen Amarok 3.0 V6 Extreme


In [14]:
df_mercado_libre_links[df_mercado_libre_links['type'] == '0km']

,type,link_web,modelo
0,0km,https://auto.mercadolibre.com.ar/MLA-147437521...,Peugeot 208 1.2 New Like
1,0km,https://auto.mercadolibre.com.ar/MLA-138489725...,Volkswagen Taos 1.4 250 Tsi Comfortline
2,0km,https://auto.mercadolibre.com.ar/MLA-148042873...,Volkswagen Amarok 3.0 V6 Extreme
3,0km,https://auto.mercadolibre.com.ar/MLA-148042856...,Volkswagen Amarok 3.0 V6 Cd Highline
4,0km,https://auto.mercadolibre.com.ar/MLA-138449856...,Volkswagen Amarok 3.0 V6 Cd Comfortline
...,...,...,...
521,0km,https://auto.mercadolibre.com.ar/MLA-138196186...,Mercedes-benz Sprinter 316 Mixto 4+1 3665 Te
522,0km,https://auto.mercadolibre.com.ar/MLA-145331258...,Chery Tiggo 4 Pro 1.5 Comfort Mt
523,0km,https://auto.mercadolibre.com.ar/MLA-144144665...,Toyota Hilux 2.8 Cd Srx 177cv 4x4 At
524,0km,https://auto.mercadolibre.com.ar/MLA-925824993...,Foton Aumark Tm1 C/d


In [15]:
def get_models_specifications(row):
    url = row['link_web']
    specs = None
    res = requests.get(url)
    soup = BeautifulSoup(res.text, "html.parser")
    tree = html.fromstring(res.content)
    
    #modelo = soup.find('h1', class_='ui-pdp-title').text.strip()
    anio = soup.find('span', class_='ui-pdp-subtitle').text.strip()
    precio = soup.find('span', class_='andes-money-amount__fraction').text.strip()
    
    list_specs = soup.find_all('span', class_='andes-table__column--value')
    default_value = "Valor por defecto"

    if len(list_specs) != 0:
        specs = [span.text.strip() for span in list_specs]
        
#     print(specs)
    
    if specs and len(specs) != 0 and len(anio) != 0 and len(precio) != 0:
        return (
            anio[0:4], 
            precio, 
            specs[3] if len(specs) > 3 else "N/A", #VERSION
            specs[4] if len(specs) > 4 else "N/A", #COLOR
            specs[5] if len(specs) > 5 else "N/A", #COMBUSTIBLE
            specs[6] if len(specs) > 6 else "N/A", #PUERTAS
            specs[7] if len(specs) > 7 else "N/A", #TRANSMISION
            specs[8] if len(specs) > 8 else "N/A", #MOTOR
            specs[9] if len(specs) > 9 else "N/A" #VEHICULO
        )
    
    return ( 
        anio[0:4], 
        precio, 
        "N/A", #VERSION
        "N/A", #COLOR
        "N/A", #COMBUSTIBLE
        "N/A", #PUERTAS
        "N/A", #TRANSMISION
        "N/A", #MOTOR,
        "N/A" #VEHICULO
    )

In [16]:
# df_modelos_okm = pd.DataFrame(modelos_okm)
df_modelos_okm = df_mercado_libre_links[df_mercado_libre_links['type'] == '0km']
df_modelos_okm

,type,link_web,modelo
0,0km,https://auto.mercadolibre.com.ar/MLA-147437521...,Peugeot 208 1.2 New Like
1,0km,https://auto.mercadolibre.com.ar/MLA-138489725...,Volkswagen Taos 1.4 250 Tsi Comfortline
2,0km,https://auto.mercadolibre.com.ar/MLA-148042873...,Volkswagen Amarok 3.0 V6 Extreme
3,0km,https://auto.mercadolibre.com.ar/MLA-148042856...,Volkswagen Amarok 3.0 V6 Cd Highline
4,0km,https://auto.mercadolibre.com.ar/MLA-138449856...,Volkswagen Amarok 3.0 V6 Cd Comfortline
...,...,...,...
521,0km,https://auto.mercadolibre.com.ar/MLA-138196186...,Mercedes-benz Sprinter 316 Mixto 4+1 3665 Te
522,0km,https://auto.mercadolibre.com.ar/MLA-145331258...,Chery Tiggo 4 Pro 1.5 Comfort Mt
523,0km,https://auto.mercadolibre.com.ar/MLA-144144665...,Toyota Hilux 2.8 Cd Srx 177cv 4x4 At
524,0km,https://auto.mercadolibre.com.ar/MLA-925824993...,Foton Aumark Tm1 C/d


In [17]:
df_modelos_okm['anio'], df_modelos_okm['precio'], df_modelos_okm['version'], df_modelos_okm['color'], df_modelos_okm['combustible'], df_modelos_okm['puertas'] , df_modelos_okm['transmision'] , df_modelos_okm['motor'] , df_modelos_okm['vehiculo'] = zip(*df_modelos_okm.apply(get_models_specifications, axis=1))
# df_modelos_okm['MODELO_'], df_modelos_okm['anio'], df_modelos_okm['precio'] = zip(*df_modelos_okm.apply(get_models_okm_specifications, axis=1))

C:\Users\PC\AppData\Local\Temp\ipykernel_9836\4289120513.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_modelos_okm['anio'], df_modelos_okm['precio'], df_modelos_okm['version'], df_modelos_okm['color'], df_modelos_okm['combustible'], df_modelos_okm['puertas'] , df_modelos_okm['transmision'] , df_modelos_okm['motor'] , df_modelos_okm['vehiculo'] = zip(*df_modelos_okm.apply(get_models_specifications, axis=1))
C:\Users\PC\AppData\Local\Temp\ipykernel_9836\4289120513.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

In [18]:
df_modelos_okm

,type,link_web,modelo,anio,precio,version,color,combustible,puertas,transmision,motor,vehiculo
0,0km,https://auto.mercadolibre.com.ar/MLA-147437521...,Peugeot 208 1.2 New Like,2023,3.120.000,N/A,N/A,N/A,N/A,N/A,N/A,N/A
1,0km,https://auto.mercadolibre.com.ar/MLA-138489725...,Volkswagen Taos 1.4 250 Tsi Comfortline,2023,17.700.000,N/A,N/A,N/A,N/A,N/A,N/A,N/A
2,0km,https://auto.mercadolibre.com.ar/MLA-148042873...,Volkswagen Amarok 3.0 V6 Extreme,2023,26.000.000,N/A,N/A,N/A,N/A,N/A,N/A,N/A
3,0km,https://auto.mercadolibre.com.ar/MLA-148042856...,Volkswagen Amarok 3.0 V6 Cd Highline,2023,24.000.000,3.0 V6 Cd Highline,Blanco,Diésel,4,Automática,3.0,Pick-Up
4,0km,https://auto.mercadolibre.com.ar/MLA-138449856...,Volkswagen Amarok 3.0 V6 Cd Comfortline,2023,20.350.000,N/A,N/A,N/A,N/A,N/A,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
521,0km,https://auto.mercadolibre.com.ar/MLA-138196186...,Mercedes-benz Sprinter 316 Mixto 4+1 3665 Te,2022,86.800,N/A,N/A,N/A,N/A,N/A,N/A,N/A
522,0km,https://auto.mercadolibre.com.ar/MLA-145331258...,Chery Tiggo 4 Pro 1.5 Comfort Mt,2023,18.900,N/A,N/A,N/A,N/A,N/A,N/A,N/A
523,0km,https://auto.mercadolibre.com.ar/MLA-144144665...,Toyota Hilux 2.8 Cd Srx 177cv 4x4 At,2023,400.000,N/A,N/A,N/A,N/A,N/A,N/A,N/A
524,0km,https://auto.mercadolibre.com.ar/MLA-925824993...,Foton Aumark Tm1 C/d,2021,12.800.000,N/A,N/A,N/A,N/A,N/A,N/A,N/A


In [19]:
df_modelos_usados = df_mercado_libre_links[df_mercado_libre_links['type'] == 'usado']
df_modelos_usados

,type,link_web,modelo
526,usado,https://auto.mercadolibre.com.ar/MLA-148356516...,Chevrolet Onix 1.4 Ltz Mt 98cv
527,usado,https://auto.mercadolibre.com.ar/MLA-138488314...,Renault Fluence 2.0 Privilege Cvt
528,usado,https://auto.mercadolibre.com.ar/MLA-138490950...,Renault Fluence 2.0 Privilege Cvt
529,usado,https://auto.mercadolibre.com.ar/MLA-138490876...,Ford Focus Ii 2.0 Exe Sedan Ghia
530,usado,https://auto.mercadolibre.com.ar/MLA-138493666...,Ford Ka 1.5 S 4 P
...,...,...,...
1057,usado,https://auto.mercadolibre.com.ar/MLA-137853497...,Peugeot 308 1.6 Allure Nav 115cv
1058,usado,https://auto.mercadolibre.com.ar/MLA-144093409...,Chevrolet Cruze 1.4 Ltz At Sedan
1059,usado,https://auto.mercadolibre.com.ar/MLA-144740810...,Jeep Renegade 1.8 Sport
1060,usado,https://auto.mercadolibre.com.ar/MLA-137762034...,Volkswagen Amarok 3.0 V6 Extreme


In [20]:
df_modelos_usados['anio'], df_modelos_usados['precio'], df_modelos_usados['version'], df_modelos_usados['color'], df_modelos_usados['combustible'], df_modelos_usados['puertas'] , df_modelos_usados['transmision'] , df_modelos_usados['motor'] , df_modelos_usados['vehiculo'] = zip(*df_modelos_usados.apply(get_models_specifications, axis=1))

C:\Users\PC\AppData\Local\Temp\ipykernel_9836\744551213.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_modelos_usados['anio'], df_modelos_usados['precio'], df_modelos_usados['version'], df_modelos_usados['color'], df_modelos_usados['combustible'], df_modelos_usados['puertas'] , df_modelos_usados['transmision'] , df_modelos_usados['motor'] , df_modelos_usados['vehiculo'] = zip(*df_modelos_usados.apply(get_models_specifications, axis=1))
C:\Users\PC\AppData\Local\Temp\ipykernel_9836\744551213.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/ind

In [21]:
df_modelos_usados

,type,link_web,modelo,anio,precio,version,color,combustible,puertas,transmision,motor,vehiculo
526,usado,https://auto.mercadolibre.com.ar/MLA-148356516...,Chevrolet Onix 1.4 Ltz Mt 98cv,2013,4.950.000,N/A,N/A,N/A,N/A,N/A,N/A,N/A
527,usado,https://auto.mercadolibre.com.ar/MLA-138488314...,Renault Fluence 2.0 Privilege Cvt,2011,4.200.000,N/A,N/A,N/A,N/A,N/A,N/A,N/A
528,usado,https://auto.mercadolibre.com.ar/MLA-138490950...,Renault Fluence 2.0 Privilege Cvt,2011,4.500.000,N/A,N/A,N/A,N/A,N/A,N/A,N/A
529,usado,https://auto.mercadolibre.com.ar/MLA-138490876...,Ford Focus Ii 2.0 Exe Sedan Ghia,2011,3.439.520,2.0 Exe Sedan Ghia,Blanco,Nafta,4,Manual,2.0,Sedán
530,usado,https://auto.mercadolibre.com.ar/MLA-138493666...,Ford Ka 1.5 S 4 P,2018,5.929.520,N/A,N/A,N/A,N/A,N/A,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
1057,usado,https://auto.mercadolibre.com.ar/MLA-137853497...,Peugeot 308 1.6 Allure Nav 115cv,2014,4.100.000,N/A,N/A,N/A,N/A,N/A,N/A,N/A
1058,usado,https://auto.mercadolibre.com.ar/MLA-144093409...,Chevrolet Cruze 1.4 Ltz At Sedan,2018,8.268.500,N/A,N/A,N/A,N/A,N/A,N/A,N/A
1059,usado,https://auto.mercadolibre.com.ar/MLA-144740810...,Jeep Renegade 1.8 Sport,2023,9.490.000,1.8 Sport,Negro,Nafta,5,Manual,1.8,SUV
1060,usado,https://auto.mercadolibre.com.ar/MLA-137762034...,Volkswagen Amarok 3.0 V6 Extreme,2021,14.500.000,N/A,N/A,N/A,N/A,N/A,N/A,N/A


In [22]:
df_mercado_libre_modelos = pd.concat([df_modelos_okm, df_modelos_usados], ignore_index=True)
df_mercado_libre_modelos.to_csv('../CSV/df_mercado_libre_modelos.csv', index=False)
df_mercado_libre_modelos

,type,link_web,modelo,anio,precio,version,color,combustible,puertas,transmision,motor,vehiculo
0,0km,https://auto.mercadolibre.com.ar/MLA-147437521...,Peugeot 208 1.2 New Like,2023,3.120.000,N/A,N/A,N/A,N/A,N/A,N/A,N/A
1,0km,https://auto.mercadolibre.com.ar/MLA-138489725...,Volkswagen Taos 1.4 250 Tsi Comfortline,2023,17.700.000,N/A,N/A,N/A,N/A,N/A,N/A,N/A
2,0km,https://auto.mercadolibre.com.ar/MLA-148042873...,Volkswagen Amarok 3.0 V6 Extreme,2023,26.000.000,N/A,N/A,N/A,N/A,N/A,N/A,N/A
3,0km,https://auto.mercadolibre.com.ar/MLA-148042856...,Volkswagen Amarok 3.0 V6 Cd Highline,2023,24.000.000,3.0 V6 Cd Highline,Blanco,Diésel,4,Automática,3.0,Pick-Up
4,0km,https://auto.mercadolibre.com.ar/MLA-138449856...,Volkswagen Amarok 3.0 V6 Cd Comfortline,2023,20.350.000,N/A,N/A,N/A,N/A,N/A,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
1057,usado,https://auto.mercadolibre.com.ar/MLA-137853497...,Peugeot 308 1.6 Allure Nav 115cv,2014,4.100.000,N/A,N/A,N/A,N/A,N/A,N/A,N/A
1058,usado,https://auto.mercadolibre.com.ar/MLA-144093409...,Chevrolet Cruze 1.4 Ltz At Sedan,2018,8.268.500,N/A,N/A,N/A,N/A,N/A,N/A,N/A
1059,usado,https://auto.mercadolibre.com.ar/MLA-144740810...,Jeep Renegade 1.8 Sport,2023,9.490.000,1.8 Sport,Negro,Nafta,5,Manual,1.8,SUV
1060,usado,https://auto.mercadolibre.com.ar/MLA-137762034...,Volkswagen Amarok 3.0 V6 Extreme,2021,14.500.000,N/A,N/A,N/A,N/A,N/A,N/A,N/A


In [23]:
fecha_hora_actual = datetime.datetime.now()
# Formatear la fecha y hora en un formato específico
formato_personalizado = "%Y-%m-%d %H:%M:%S"  # Por ejemplo, "2023-08-25 15:30:00"
fecha_hora_formateada = fecha_hora_actual.strftime(formato_personalizado)
print("Fecha y hora:", fecha_hora_formateada)

Fecha y hora: 2023-08-27 19:25:19
